# Emotion Classification

In [ ]:
!pip install -q datasets wandb

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer, TrainingArguments
    )
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import wandb
import torch
from huggingface_hub import login

In [ ]:
!wandb login

In [ ]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6).to('cuda')

In [ ]:
dataset = load_dataset('KittiphopKhankaew/aina-emotion-classification')
dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

In [ ]:
train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['test']

In [ ]:
wandb.init(project="aina")
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # Number of epochs
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=1,
    save_strategy="steps",
    save_total_limit=2,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    gradient_accumulation_steps=2,
    seed=42
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
        "f1": f1_score(labels, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./aina-emotion-classification")

In [ ]:
login()

In [ ]:
trainer.push_to_hub('KittiphopKhankaew/aina-emotion-classification')